In [1]:
import pandas as pd 
import numpy as np
import random
from scipy.sparse.linalg import cg

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/final_users.csv")
df.head()

,ID,Name,Rating,Name_ID,New_ID
0,1,Siddhartha,5,1,1
1,1,"The Hunger Games (The Hunger Games, #1)",5,2,1
2,1,The Authoritative Calvin and Hobbes: A Calvin ...,5,3,1
3,1,The Return of the Indian (The Indian in the Cu...,5,4,1
4,1,Dark Apprentice (Star Wars: The Jedi Academy T...,3,5,1


In [4]:
print(df['Name'].nunique()) #returns the unique users
print(df['ID'].nunique())   #returns the unique ids

11057
3534


In [5]:
def nmae(tes,f):    #function to compute normalised mean absolute error
  mae_sum =0          
  ratings_range = 4   
  # s=0                
  n=0     # length     
  for i in range(3534):
    for j in range(11057):
      if tes[i][j]>0:
        n+=1
        mae_sum += np.abs(tes[i][j]-f[i][j])  
        # s+=tes[i][j]
  nmae = mae_sum/(ratings_range*n)
  return nmae

In [6]:
def entire_matrix_creation(df):   # function to create the user-item matrix
  user_item_matrix=np.zeros((3534,11057))
  for i in range(len(df)):
      user_item_matrix[df["New_ID"][i]-1][df["Name_ID"][i]-1] = df["Rating"][i]
  return user_item_matrix

In [7]:
def get_coordinates(matrix):
    coordinates = []
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            if matrix[i][j] > 0:
                coordinates.append((i, j))
    return coordinates

def split_list(list_to_split, ratio=0.7):
    split_index = int(len(list_to_split) * ratio)

    random.shuffle(list_to_split)

    train = list_to_split[:split_index]
    test = list_to_split[split_index:]
    
    return train, test

In [8]:
def train_test_creation(train_coordinates,test_coordinates,data):
  train = np.zeros_like(data)
  test = np.zeros_like(data)
  for i in train_coordinates:
    train[i[0]][i[1]] = data[i[0]][i[1]]

  for i in test_coordinates:
    test[i[0]][i[1]] = data[i[0]][i[1]]
  
  return train, test

### Multiplicative Updates

In [10]:
def matrix_factorization(iterations,y,u,v,r):
  x = np.dot(u,v)
  for i in range(iterations):
    b = x +y - r*(np.dot(u,v))
    for j in range(10):
      u*= np.divide(np.dot(b,v.T),np.dot(np.dot(u,v),v.T))
      v*= np.divide(np.dot(u.T,b),np.dot(np.dot(u.T,u),v))
    x = np.dot(u,v)
  return x

In [11]:
data = entire_matrix_creation(df)

In [12]:
kk = get_coordinates(data)
train_coordinates, test_coordinates = split_list(kk,0.85)
train, test = train_test_creation(train_coordinates,test_coordinates,data)

In [28]:
r = 1*(train>0)
u = np.random.randint(low=0, high=6, size=(3534, 70))*0.1
v = np.random.randint(low=0, high=6, size=(70, 11057))*0.1

In [29]:
f = matrix_factorization(500, train, u, v, r)
print(nmae(test,f))

0.20470791591124335


In [30]:
data

array([[5., 5., 5., ..., 0., 0., 0.],
       [0., 4., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [31]:
f

array([[4.60996879, 3.83047743, 5.28152522, ..., 4.8918093 , 4.62225343,
        3.8474758 ],
       [3.64095239, 4.8555123 , 4.97809635, ..., 4.30031683, 4.07950517,
        4.76757791],
       [4.72304556, 5.06747926, 5.19242865, ..., 4.96201728, 5.32830976,
        5.02823689],
       ...,
       [3.58374745, 4.92302828, 4.40591816, ..., 4.51605563, 4.31840826,
        5.12141052],
       [3.77776847, 5.36827428, 4.96972443, ..., 5.21001551, 4.95639964,
        4.79659294],
       [4.00479863, 3.77131526, 4.45578079, ..., 4.73659833, 4.98002966,
        4.75863331]])

In [32]:
print(np.max(f))

8.510228302824407


In [34]:
import pickle
pickle.dump(u, open('/content/drive/MyDrive/Colab Notebooks/u70.pkl', 'wb'))
pickle.dump(v, open('/content/drive/MyDrive/Colab Notebooks/v70.pkl', 'wb'))
pickle.dump(f, open('/content/drive/MyDrive/Colab Notebooks/f70.pkl', 'wb'))